### CNN을 이용하여 강아지와 고양이 구별하기

라이브러리 설치와 이미지출력

In [ ]:
# pip install pillow
# pip install matplotlib

from matplotlib import pyplot
from matplotlib.image import imread

image = imread('./Petimages/train/dog/1.jpg')
pyplot.imshow(image)
pyplot.show()

CNN 모델 생성

![Alt text](image-1.png)

In [ ]:
from tensorflow.keras import models, layers

train_dir = './Petimages/train'
test_dir = './Petimages/test'

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=(128,128,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dense(units=512, activation='relu'))
model.add(layers.Dense(units=1, activation='sigmoid'))

model.summary()

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

이미지 전처리

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 0에서 255사이의 픽셀값을 0.0-1.0사이의 실수 변환 등 이미지 전처리
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2,
  zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,                      
    target_size=(128, 128), 
    batch_size=20,
    class_mode = 'binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=20,
    class_mode = 'binary')

훈련


In [ ]:
# 훈련 데이터로는 train_generator가 건네주는 데이터 사용
# 검증 데이터로는 test_generator가 전달하는 데이터 사용
# steps_per_epoch : 훈련에 사용할 배치의 개수. len(train) // batch_size
# validation_steps : 테스트에 사용할 배치의 개수. len(test) // bacth_size
history = model.fit_generator(
    train_generator, steps_per_epoch = 100, epochs=10, 
    validation_data=test_generator, validation_steps=5)

학습 결과 그래프 표시

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

(추가) 다른 이미지로 테스트

In [ ]:
# OpenCV 설치 - pip install opencv-python

import cv2 as cv
import matplotlib.pyplot as plt

image = cv.imread('dog1.jpg')

image = cv.resize(image, (128, 128))
image = image.astype('float32')
image = 255-image
image /= 255.0

plt.imshow(image.reshape(128, 128, 3))
plt.show()

pred = model.predict(image.reshape(1, 128, 128, 3))
print("출력 : ", pred, )    # 출력은 시그모이드 값으로 나온다.

if pred[0] < 0.5 :  # 데이터가 작기 때문에 모델 정확도가 높지 않음
    print("강아지")
else : 
    print("고양이")